In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

#load Bitcoin data
df = pd.read_csv("btc_data.csv")

#using the 'Close' price column
close_prices = df['Close'].values.reshape(-1, 1)

#Normalize
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(close_prices)

#split into train and test
train, test = train_test_split(dataset, test_size=0.5, shuffle=False)

#function to create datasets
def create_dataset(data, look_back=240):
    X, Y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data[i:(i + look_back), 0])
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 240
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

#reshape for LSTM input
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#LSTM Model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(1, look_back)))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))

#compile and train
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, trainY, epochs=1000, batch_size=240, verbose=1)

#predict
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

#inverse transform
trainPredict = scaler.inverse_transform(trainPredict)
trainY_inv = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY_inv = scaler.inverse_transform([testY])

#RMSE
trainScore = math.sqrt(mean_squared_error(trainY_inv[0], trainPredict[:, 0]))
testScore = math.sqrt(mean_squared_error(testY_inv[0], testPredict[:, 0]))

print(f"Bitcoin LSTM Model - Train RMSE: {trainScore:.2f}")
print(f"Bitcoin LSTM Model - Test RMSE: {testScore:.2f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.0303
Epoch 2/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0110 
Epoch 3/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0080 
Epoch 4/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0070 
Epoch 5/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0054 
Epoch 6/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0054
Epoch 7/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0042
Epoch 8/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0032 
Epoch 9/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0025 
Epoch 10/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0024
Epoch 11/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0024
Epoch 12/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0019 
Epoch 13/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0020 
Epoch 14/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0016 
Epoch 15/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0016
E